In [1]:
import pandas as pd
from packages import wrappers, ta

In [2]:
btcusd_df = wrappers.get_df_from_cryptowatch()

,Open,High,Low,Close,Volume,QuoteVolume
Date,,,,,,
2013-10-08,123.61,123.61000,123.61,123.61,0.100000,0.000000e+00
2013-10-09,123.61,124.19000,123.90,124.18,3.991600,0.000000e+00
2013-10-10,124.18,124.01687,123.84,123.84,2.823000,0.000000e+00
2013-10-11,123.84,125.86000,125.85,125.86,2.000000,0.000000e+00
2013-10-13,125.86,127.50000,127.00,127.00,4.000000,0.000000e+00
...,...,...,...,...,...,...
2021-08-10,43845.60,46484.10000,42827.70,46307.10,5654.101582,2.560590e+08
2021-08-11,46300.80,46688.00000,44666.00,45595.10,3102.043540,1.412968e+08
2021-08-12,45606.90,46736.40000,45373.50,45571.70,3504.532810,1.616380e+08


In [ ]:
ta.add_ATR(btcusd_df, inplace=True)
ta.add_OBV(btcusd_df, inplace=True)
btcusd_df